In [5]:
%load_ext autoreload
%autoreload 2

In [4]:
# dependencies`
from facenet_pytorch import MTCNN
import torch
import numpy as np
import itertools as it
import mmcv, cv2
import matplotlib.pyplot as plt

from IPython.display import Video
from scipy.linalg import block_diag
from scipy.optimize import linear_sum_assignment
from PIL import Image, ImageDraw
from IPython import display
from pathlib import Path
from time import time
from scipy.spatial.distance import mahalanobis
from JAVER.track_tools.frame_handler import FrameHandler
from JAVER.track_tools.face_tracker import FaceTracker

import logging
for name in logging.root.manager.loggerDict:
    logging.getLogger(name).setLevel("CRITICAL") 
    
from JAVER.logger import create_logger
logger = create_logger(level='DEBUG')
logger.setLevel('INFO')

ModuleNotFoundError: No module named 'JAVER.logger'

In [24]:
video_path = '../../tests/data/Elon Musk/multi_person_jre.mp4'

In [25]:
# setup face tracker
p = Path("../../tests/data/Elon Musk/")
ref_paths = list(p.glob("inliers/*"))

face_tracker = FaceTracker(
    image_size=160, 
    ref_paths=ref_paths, 
    batch_size=8, 
    step_large=150, 
    step_small=10
)

faces_dict = face_tracker.detect(video_path)

[2020-04-19 18:49:08,597] INFO @ line 393: Rollback to 0
[2020-04-19 18:49:08,608] WARNING @ line 280: Step size too big!!
[2020-04-19 18:50:58,605] INFO @ line 421: Finished Tracking


In [26]:
tracks = face_tracker.associate(faces_dict)

AssertionError: 

In [21]:
from moviepy.editor import VideoFileClip

def moving_crop(clip,fx,fy, size=(160,160)):
    def scale(im, nR, nC):
        nR0 = len(im)     # source number of rows 
        nC0 = len(im[0])  # source number of columns 
        return np.asarray([[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
                 for c in range(nC)] for r in range(nR)])

    def fl(gf,t):
        im = gf(t)

        x1,x2 = list(map(int, fx(int(t*25))))
        y1,y2 = list(map(int, fy(int(t*25))))

        return scale(im[y1:y2, x1:x2], 160, 160)
    
    return clip.fl(fl)

In [30]:
out_path = 'ani.mp4'
clip = VideoFileClip(video_path)
subclip = clip.subclip(0, 10)

track = tracks[0]

x1s, y1s, x2s, y2s = track.get_bboxes()

fx = lambda t: (x1s[t], x2s[t])
fy = lambda t: (y1s[t], y2s[t])

subclip = subclip.fx(moving_crop, fx=fx, fy=fy)

subclip.write_videofile(out_path, fps=25)

t:   6%|▌         | 261/4525 [00:35<04:23, 16.18it/s, now=None]

Moviepy - Building video ani.mp4.
MoviePy - Writing audio in aniTEMP_MPY_wvf_snd.mp3



t:   6%|▌         | 261/4525 [00:36<04:23, 16.18it/s, now=None]    

MoviePy - Done.
Moviepy - Writing video ani.mp4




t:   6%|▌         | 261/4525 [00:52<04:23, 16.18it/s, now=None][A

Moviepy - Done !
Moviepy - video ready ani.mp4


In [23]:
Video(out_path, width=160)